# Obsługa ROS Service

## Używanie komend z terminala

### Wprowadzenie

ROS Service jest kolejnym sposobem komunikacji. Zaletą serwisów jest możliwość wysłania zapytania do serwera. Podobnie jak dla ROS topic należy znać format wiadomości. Serwisy w danej paczce ROS przechowywane są w katalogu srv, a rozszerzenie serwisu to **.srv**. 

#### Struktura wiadomości serwisowej

Wyróżnia się podział na format wiadomości:
- wysyłany przez klienta (górna część wiadomości nad znakiem **---**)
- odpowiedź serwera (dolna część wiadomości pod znakiem **---**)

Z lewej strony należy podać typ wiadomości ROS. Mogą być one bardziej złożone i składać się z już utworzonych
wiadomości (ROS msg). Z prawej strony podawana jest nazwa pola.

Typ serwisu to **nazwa paczki + nazwa serwisu.srv**. Wyświetlenie przykładowego serwisu znajdującego się w paczce tsr_materials:

In [1]:
!rossrv show pkg_tsr/Task

float64 x
float64 y
---
string result



#### Wyświetlanie listy aktualnie dostępnych serwisów

In [2]:
!rosservice list

/PositionUpdater_5115_1619277478157/get_loggers
/PositionUpdater_5115_1619277478157/set_logger_level
/clear
/kill
/reset
/rosout/get_loggers
/rosout/set_logger_level
/rostopic_4284_1619275916185/get_loggers
/rostopic_4284_1619275916185/set_logger_level
/spawn
/teleop_turtle/get_loggers
/teleop_turtle/set_logger_level
/turtle1/set_pen
/turtle1/teleport_absolute
/turtle1/teleport_relative
/turtlesim/get_loggers
/turtlesim/set_logger_level


#### Wyświetlanie dostępnej pomocy dla polecenia rosservice.

In [2]:
!rosservice --help

Commands:
	rosservice args	print service arguments
	rosservice call	call the service with the provided args
	rosservice find	find services by service type
	rosservice info	print information about service
	rosservice list	list active services
	rosservice type	print service type
	rosservice uri	print service ROSRPC uri

Type rosservice <command> -h for more detailed usage, e.g. 'rosservice call -h'



### Dostępne serwisy dla turtlesim node

W podstawowym zakresie dostępne są następujące serwisy bez względu na robota:
- /clear - wyczyszczenie narysowanych ścieżek
- /kill - usunięcie robota
- /reset - reset środowiska do stanu początkowego
- /spawn - utworzenie nowego robota

Dla pojedynczego utworzonego robota w przestrzeni nazw na przykładzie **turtle1** dostępne są nastpujące serwisy:
- /turtle1/set_pen - ustawienie koloru pędzla do rysowania
- /turtle1/teleport_absolute - natychmiastowe przeniesienie robota do wskazanej lokalizacji
- /turtle1/teleport_relative - przeniesienie robota, współrzędne podawane w układzie robota

Wyświetlenie informacji o serwisie "clear". Informuje o nazwie noda z którego pochodzi serwis, typie wiadomości i przyjmowanych argumentach.

In [3]:
!rosservice info clear

Node: /turtlesim
URI: rosrpc://localhost:54857
Type: std_srvs/Empty
Args: 


#### Zadanie 1
Wyświetlić informacje dla pozostałych serwisów

In [3]:
!rosservice info kill

Node: /turtlesim
URI: rosrpc://localhost:37269
Type: turtlesim/Kill
Args: name


In [4]:
!rosservice info reset

Node: /turtlesim
URI: rosrpc://localhost:37269
Type: std_srvs/Empty
Args: 


In [5]:
!rosservice info spawn

Node: /turtlesim
URI: rosrpc://localhost:37269
Type: turtlesim/Spawn
Args: x y theta name


In [6]:
!rosservice info turtle1/set_pen

Node: /turtlesim
URI: rosrpc://localhost:37269
Type: turtlesim/SetPen
Args: r g b width off


In [7]:
!rosservice info turtle1/teleport_absolute

Node: /turtlesim
URI: rosrpc://localhost:37269
Type: turtlesim/TeleportAbsolute
Args: x y theta


In [8]:
!rosservice info turtle1/teleport_relative

Node: /turtlesim
URI: rosrpc://localhost:37269
Type: turtlesim/TeleportRelative
Args: linear angular


### Wywołanie serwisu

Do wywołania serwisu używamy rosservice call a następnie w kolejności podajemy argumenty

*rosservice call argument1 argument2 ...*

**Uwaga! Współrzędne x,y powinny być z przedziału <0,11>**

Tworzenie nowego robota.

In [11]:
!rosservice call spawn 5 5 1 NowyRobot

name: "NowyRobot"


#### Zadanie 2
Utworzyć dodatkowe 2 roboty o różnych nazwach w różnych miejscach

In [12]:
!rosservice call spawn 2 2 2 Arcio
!rosservice call spawn 10 2 2 Benek

name: "Arcio"
name: "Benek"


#### Zadanie 3
Narysować dowolnym robotem kwadrat wykorzystując serwis /*nazwa_robota*/teleport_absolute

In [14]:
!rosservice call /Benek/teleport_absolute 10 10 2
!rosservice call /Benek/teleport_absolute 2 10 2
!rosservice call /Benek/teleport_absolute 2 2 2
!rosservice call /Benek/teleport_absolute 10 2 2

#### Zadanie 4
Usunąć wszystkie roboty. 

In [15]:
!rosservice call kill turtle1
!rosservice call kill NowyRobot
!rosservice call kill Benek
!rosservice call kill Arcio

## Używanie serwisów w Pythonie - klient

In [4]:
import rospy
rospy.init_node("serwis_node_test")

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


ROSInitException: Failed to initialize time. Please check logs for additional details

Serwis */clear* do czyszczenia mapy, bez argumentów. Po stronie klienta do obsługi serwisu używamy *ServiceProxy* z biblioteki *rospy*. Jako pierwszy argument podawana jest nazwa serwisu z którego ma być odebrana odpowiedź, a jako drugi argument podawany jest typ serwisu.

In [18]:
#import serwisu
from std_srvs.srv import Empty

In [19]:
clear_map = rospy.ServiceProxy('clear', Empty)
clear_map()

Czysczenie przestrzeni z robotów */reset*

In [20]:
reset_sim_state = rospy.ServiceProxy('reset', Empty)
reset_sim_state()

Używanie serwisu z argumentami na przykładzie */spawn*.  Kolejne argumenty polecenia podajemy po przecinku.

In [22]:
from turtlesim.srv import Spawn

In [23]:
create_new_robot = rospy.ServiceProxy('spawn', Spawn)
create_new_robot(3, 3, 0, "t8")

name: "t8"

#### Zadanie 5
Ustawić kolor pędzla na czerwony dla nowo utworzonego robota używając kodu w Pythonie

In [27]:
# UZUPEŁNIĆ
from turtlesim.srv import SetPen
set_pen = rospy.ServiceProxy('/t8/set_pen',SetPen)
set_pen(255,0,0,20,0)

#### Zadanie 6
Tak jak wcześniej narysować kwadrat, ale wykorzystując program w Pythonie.

In [28]:
# UZUPEŁNIĆ
from turtlesim.srv import TeleportAbsolute
tp = rospy.ServiceProxy('/t8/teleport_absolute',TeleportAbsolute)
tp(3,8,0)
tp(8,8,0)
tp(8,3,0)
tp(3,3,0)

## Używanie serwisów w Pythonie - serwer

In [29]:
!rosservice call reset

Po stronie serwera używamy obiektu *Service* z biblioteki *rospy*. W kolejności podajemy 3 następujące argumenty:
    - nazwa serwisu
    - typ serwisu
    - nazwa funkcji, która ma zostać wywołan po pojawieniu się żądania od klienta na tym serwisie

In [30]:
from pkg_tsr.srv import Task
from turtlesim.srv import TeleportAbsolute
from std_srvs.srv import Empty

In [31]:
def draw_square_function(req):
    width = 5
    print("Init pose x=%s, y=%s" % (req.x, req.y))
    set_pose = rospy.ServiceProxy('/turtle1/teleport_absolute', TeleportAbsolute)
    clear_map = rospy.ServiceProxy('clear', Empty)
    set_pose(req.x,req.y,0)
    clear_map()
    set_pose(req.x,req.y + width,0)
    set_pose(req.x + width,req.y + width,0)
    set_pose(req.x + width, req.y,0)
    set_pose(req.x, req.y,0)
    return "finished"

In [32]:
s = rospy.Service('robot_teleport', Task, draw_square_function)

#### UWAGA
Wyłączenie serwisu - gdy wprowadzone zostaną jakieś zmiany w funkcji serwisowej, aby nie restartować Kernela Jupyter Notebook, można poniższą metodą shutdown() zatrzymać działający serwis.

In [35]:
s.shutdown()

#### Wynik działania powyższego serwisu

In [33]:
draw_square = rospy.ServiceProxy('robot_teleport', Task)
resp = draw_square(1, 2)

Init pose x=1.0, y=2.0


Odczyt otrzymanej wartości z serwera

In [34]:
resp.result

'finished'

#### Zadanie 7
1. Należy uzupełnić metodę **go_to_goal()** odpowiadającą za dojazd do punktu o podanych współrzędnych. W klasie Robot znajduje się serwis odpowiedzialny za wywołanie tej metody. Serwis korzysta z typu **tsr_materials/Task.srv**. W zapytaniu od klienta w polach x,y należy przesłać współrzędne celu. W odpowiedzi na zapytanie klienta należy odesłać informację o zakończeniu działania. Metoda powinna wysyłać odpowiednie prędkości sterujące robotem (postępową i obrotową)na topicu **/nazwa_robot/cmd_vel**. Należy ustawić wartość zmiennej od tolerancji dojazdu do celu (np.0.05) i skorzystać z niej przy dojeżdżaniu do celu. Dodatkowo powinny zostać uwzględnione ograniczenia prędkości (np. 0.3). 
2. Analogicznie do metody **go_to_goal** dołożyć inną metodę umożliwiającą dojazd do wskazanego punktu i rozpoczęcie ruchu po dowolnej krzywej np. łamana, spirala, okrąg.

In [36]:
!rosservice call reset

In [1]:
import rospy
import numpy as np
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
from geometry_msgs.msg import Twist
from pkg_tsr.srv import Task
import math

rospy.init_node("serwis_node_test")

In [21]:
class Robot():
    position = Pose()
    def __init__(self, name, x, y):
        self.name = name
        self.follow_service = rospy.Service(name+'/go_to_goal', Task, self.go_to_goal)
        self.velocity_pub = rospy.Publisher(name+'/cmd_vel', Twist, queue_size=10)
        self.pose_sub = rospy.Subscriber(name+'/pose',Pose, self.callback_pose)
        
        create_new_turtle = rospy.ServiceProxy('spawn', Spawn)
        create_new_turtle(x, y, 0, name)
        
    def go_to_goal(self, msg_srv):
        max_speed = 0.3
        tolerance = 0.05
        # UZUPEŁNIĆ - na podstawie informacji o położeniu publikować odpowiednie prędkości. Po osiągnięciu 
        # celu z zadaną tolerancją zwrócić informację, że zakończono. Do określania kierunku ruchu i wyznaczania
        # prędkości mogą przydać się funkcje:
        # - math.atan2(y,x)- zwraca 
        # - math.sqrt(x) - zwraca pierwiastek kwadratowy z x
        while abs(math.atan2(msg_srv.y - self.position.y,msg_srv.x - self.position.x) - self.position.theta)>tolerance*0.01:
            msg_vel = Twist()
            msg_vel.angular.z = np.clip(0.2*(abs(math.atan2(msg_srv.y - self.position.y,msg_srv.x - self.position.x) - self.position.theta))+0.01,0,max_speed)
            self.velocity_pub.publish(msg_vel)
        while (math.sqrt((msg_srv.x - self.position.x)**2 + (msg_srv.y - self.position.y)**2)) > tolerance:
            msg_vel = Twist()
            msg_vel.linear.x = np.clip(0.5*math.sqrt((msg_srv.x - self.position.x)**2 + (msg_srv.y - self.position.y)**2)+0.025,0,max_speed)
            self.velocity_pub.publish(msg_vel)
        
        return "finished"

    
    def callback_pose(self, data):
        self.position = data
    
    def unregister(self):
        self.follow_service.shutdown()

In [22]:
# UZUPEŁNIĆ - utworzyć klienta i zlecić zadanie dojazdu do kilku różnych punktów
robocik = Robot("Zenon5",2,2)
robocik.go_to_goal(Pose(5.5,8,0,0,0))
robocik.go_to_goal(Pose(8,2,0,0,0))
robocik.go_to_goal(Pose(2,2,0,0,0))


'finished'

In [19]:
robocik.unregister()
!rosservice call reset

In [164]:
(math.atan2(-0.001,1)+2*np.pi) % (2*np.pi)

6.282185307512919

## Dodatek

Tworzenie własnych wiadomości serwisowych. W paczce znajduje się katalog srv w którym można utworzyć dodatkowe wiadomości serwisowe. Konfiguracja w przedstawionym dodatku ogranicza się do używania typów wiadomości z poniższego linku:

http://docs.ros.org/en/melodic/api/std_msgs/html/index-msg.html

In [1]:
!rossrv show pkg_tsr/Task

float64 x
float64 y
---
string result



In [2]:
!rosmsg show std_msgs/ColorRGBA

float32 r
float32 g
float32 b
float32 a



Przykładowo rozbudowany serwis o wiadomość typu ColorRGBA może wyglądać następująco. 

Dodanie nowej wiadomości wymaga zmian w plikach konfiguracyjnych paczki. Ze względu na przyjęte uproszczenie w 
konfiguracji w paczce tsr_materials w pliku **CMakeLists.txt** w miejscu (od 57 linii):

Należy dopisać nazwę swojego serwisu jak powyżej. Zamiast **Nowy.srv** należy podać nazwę serwisu pod jakim został on zapisany w katalogu srv.

In [1]:
!rossrv show pkg_tsr/RectTask #uzupelnic nazwa nowego serwisu, jesli wszystko zostalo prawidlowo utworzone 
# powinna pojawic sie jego struktura

float64 a
float64 b
float64 start_x
float64 start_y
---
string result

